# Install Libraries if Needed (Only Run Once)

In [ ]:
# !pip install -U deepxde

# Import Libraries

In [ ]:
# Interactive Plotting

# for jupyter notebooks
%matplotlib notebook 

# for jupyter labs
# %matplotlib widget 

In [ ]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch"""
import io
import re
import os

import matplotlib.pyplot as plt
import numpy as np

import deepxde as dde
from deepxde.backend import tf

from mpl_toolkits.mplot3d import Axes3D
import sys
from scipy.integrate import odeint
import matplotlib.tri as tri
from mpl_toolkits import mplot3d
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Poisson Equation
$$
\nabla^2 u(x,y) = -1,\quad (x,y) \in \Omega, \\
u(x,y) = 0, \quad (x,y) \in \partial \Omega, \\
\Omega = [-1, 1]^2 \ \backslash \ [0,1]^2
$$

# Define PDE

In [ ]:
def pde(x, y):
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    dy_yy = dde.grad.hessian(y, x, i=1, j=1)
    return -dy_xx - dy_yy - 1

# Define Geometry

In [ ]:
geom = dde.geometry.Polygon([[0, 0], [1, 0], [1, -1], [-1, -1], [-1, 1], [0, 1]])

In [ ]:
plt.figure(figsize=(8,8))
# plt.axis('equal')

plt.fill(geom.vertices[:,0], geom.vertices[:,1])
plt.axis('square')
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.xlabel('x')
plt.ylabel('y')
plt.title('Lshape Geometry') 
plt.tight_layout()

# Define BC

In [ ]:
def boundary(_, on_boundary):
    return on_boundary

bc = dde.DirichletBC(geom, lambda x: 0, boundary)

# Define Data (PDE + BC + Geometry)

In [ ]:
data = dde.data.PDE(geom, pde, bc, num_domain=25000, num_boundary=5000, num_test=1500)

# Define Network Architecture

In [ ]:
net = dde.maps.FNN([2] + [50] * 4 + [1], "tanh", "Glorot uniform")

# Define Model (Data + Net)

In [ ]:
model = dde.Model(data, net)

# Define Paths

In [ ]:
save_dir = 'Poisson_Lshape'
model_name = 'model'
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, model_name)
ckpt_path = tf.train.latest_checkpoint(save_dir)
loss_fname = 'loss_Poisson_Lshape.dat'
train_fname = 'train_Poisson_Lshape.dat'
test_fname = 'test_Poisson_Lshape.dat'
loss_path = os.path.join(save_dir, loss_fname)
train_path = os.path.join(save_dir, train_fname)
test_path = os.path.join(save_dir, test_fname)

# ckpt_path = os.path.join(save_dir, 'checkpoint')
# model.save(model_path)

# Load Model

In [ ]:
model.compile("adam", lr=1e-3)
model.restore(ckpt_path, verbose=1)

# Train the Model
Here we train the model twice.  Once with Adam optimizer for initial training.  Then again with L-BFGS-B optimizer for finetuning.

In [ ]:
model.compile("adam", lr=0.001)
model.train(epochs=50000)
model.compile("L-BFGS")
losshistory, train_state = model.train()

# Save Model

In [ ]:
model.save(model_path)

# Plot Loss and Predictions

In [ ]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True, loss_fname=loss_path, train_fname=train_path, test_fname=test_path)

# Get Predictions

In [ ]:
def load_test_data(file='dataset/Poisson_Lshape.npz'):
    data = np.load(file)
    X_test = data['X_test']
    y_test = data['y_ref']
    return X_test, y_test

In [ ]:
file = 'dataset/Poisson_Lshape.npz'
X_test, y_test = load_test_data(file)

# Predict on Test Data

In [ ]:
y_pred = model.predict(X_test)

# Evaluate on Test Data

In [ ]:
not_nan = ~np.isnan(y_test[:,0])

print("L2 relative error:", dde.metrics.l2_relative_error(y_test[not_nan], y_pred[not_nan]))

# Plot Results

In [ ]:
not_nan

In [ ]:
X = X_test[not_nan,0]
Y = X_test[not_nan,1]
Y_test = y_test[not_nan,0]
Y_pred = y_pred[not_nan,0]


In [ ]:
triang = tri.Triangulation(X, Y)
x_triang = X[triang.triangles].mean(axis=1)
y_triang = Y[triang.triangles].mean(axis=1)
zero = np.zeros_like(x_triang)
triang.set_mask(np.greater(x_triang,zero) & np.greater(y_triang,zero))

In [ ]:
fig = plt.figure(figsize=(18,5))
plt.subplot(1,3,1)
# plt.pcolor(XX,TT, S_test, cmap='jet')
tpc = plt.tripcolor(triang, Y_test, cmap='jet', shading='gouraud')
plt.colorbar(tpc)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('Exact $u(x,y)$')
plt.axis('square')
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.tight_layout()
# plt.axis('off')

plt.subplot(1,3,2)
# plt.pcolor(XX,TT, S_pred, cmap='jet')
tpc = plt.tripcolor(triang, Y_pred, cmap='jet', shading='gouraud')
plt.colorbar(tpc)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('Predict $u(x,y)$')
plt.axis('square')
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.tight_layout()
# plt.axis('off')

plt.subplot(1,3,3)
# plt.pcolor(XX,TT, S_pred - S_test, cmap='jet')
tpc = plt.tripcolor(triang, abs(Y_pred - Y_test), cmap='jet', shading='gouraud')
plt.colorbar(tpc)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('Absolute error')
plt.axis('square')
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.tight_layout()
# plt.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize =(16, 9)) 
ax = plt.axes(projection ='3d') 
trisurf = ax.plot_trisurf(X, Y, Y_test,
                         cmap = 'jet',
                         linewidth = 0.2,
                         antialiased = True,
                         edgecolor = 'grey') 
fig.colorbar(trisurf, ax = ax, shrink = 0.5, aspect = 5)
ax.set_title('Test Values', fontweight='bold')
# Adding labels
ax.set_xlabel('x', fontweight ='bold')
ax.set_ylabel('y', fontweight ='bold')
ax.set_zlabel('u', fontweight ='bold')
plt.show()

In [ ]:
fig = plt.figure(figsize =(16, 9)) 
ax = plt.axes(projection ='3d') 
trisurf = ax.plot_trisurf(X, Y, Y_pred,
                         cmap = 'jet',
                         linewidth = 0.2,
                         antialiased = True,
                         edgecolor = 'grey') 
fig.colorbar(trisurf, ax = ax, shrink = 0.5, aspect = 5)
ax.set_title('Predictions', fontweight='bold')
# Adding labels
ax.set_xlabel('x', fontweight ='bold')
ax.set_ylabel('y', fontweight ='bold')
ax.set_zlabel('u', fontweight ='bold')
plt.show()

In [ ]:
fig = plt.figure(figsize =(16, 9)) 
ax = plt.axes(projection ='3d') 
trisurf = ax.plot_trisurf(X, Y, Y_pred - Y_test,
                         cmap = 'jet',
                         linewidth = 0.2,
                         antialiased = True,
                         edgecolor = 'grey') 
fig.colorbar(trisurf, ax = ax, shrink = 0.5, aspect = 5)
ax.set_title('Error', fontweight='bold')
# Adding labels
ax.set_xlabel('x', fontweight ='bold')
ax.set_ylabel('y', fontweight ='bold')
ax.set_zlabel('u', fontweight ='bold')
plt.show()